In [1]:
from pyquil.quil import Program
from pyquil.gates import *
from pyquil import get_qc
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import time

# QNN 2 Parametrs Evaluation

## Data Preprocessing

### Import Data

In [2]:
iris_dataset = np.loadtxt("iris.txt")
X, Y = iris_dataset[:, 0: 4], iris_dataset[:, -1]
Y[Y < 0] = 0
print(f"First sample: {X[0]}")
print(f"Label = {int(Y[0])}")

First sample: [0.4  0.75 0.2  0.05]
Label = 0


### Normalize Data

In [3]:
X_norm = 1. * np.pi * np.array([x / np.linalg.norm(x) for x in X])
print("First sample normalized:", X_norm[0])

First sample normalized: [1.43674319 2.69389348 0.71837159 0.1795929 ]


### Create Sample Data

In [4]:
def create_sample(X, Y, sample_size):
    if sample_size == 1:
        return X,Y
    X_sample,_,Y_sample,_ = train_test_split(X, Y, train_size=sample_size, stratify = Y)
    return X_sample,Y_sample

## Quantum Neural Network 

In [5]:
weights_path = 'best_weights_2p.npy'

### Reproduce Circuit

In [6]:
def feature_map(circuit, qubit_list, x):
    n_qubit = len(qubit_list)
    for i in range(n_qubit):
        circuit += H(qubit_list[i])
    for i in range(n_qubit):
        circuit += RZ(x[i], qubit_list[i])
    return circuit

def variational(circuit, qubit_list, weights):
    n_qubit = len(qubit_list)
    for i in range(2):
        circuit += RY(weights[0], qubit_list[2*i])
        circuit += RY(weights[1], qubit_list[2*i+1])
    for i in range(n_qubit - 1):
        circuit += CNOT(qubit_list[i], qubit_list[i + 1])
    for i in range(2):
        circuit += RY(weights[0], qubit_list[2*i])
        circuit += RY(weights[1], qubit_list[2*i+1])
    return circuit

In [7]:
def create_qnn_circuit(qubit_list, x, weights):
    n_qubit = len(qubit_list)
    assert n_qubit == len(x)
    #TODO assert on weights
    circuit = Program()
    for i in range(2):
        circuit = feature_map(circuit, qubit_list, x)
        circuit = variational(circuit, qubit_list, weights[0])
    ro = circuit.declare('ro', 'BIT', 1)
    circuit += MEASURE(qubit_list[0], ro[0])
    return circuit

In [8]:
def print_circuit():
    weights = np.load(weights_path)
    qubit_list = [0,1,2,3]
    circuit = create_qnn_circuit(qubit_list, np.array([0,1,2,3]), weights)
    print(circuit)

### Single x Evaluation

In [9]:
def run_circuit(circuit, qubit_list, qvm):
    n = len(qubit_list)
    shots = 10**3
    circuit.wrap_in_numshots_loop(shots)
    #qc = get_qc('3q-qvm')  # You can make any 'nq-qvm' this way for any reasonable 'n'
    qc = get_qc("Aspen-M-1", as_qvm=qvm, execution_timeout=60)
    executable = qc.compile(circuit)
    result = qc.run(executable)
    bitstrings = result.readout_data.get('ro')[:,:1]
    return (len(bitstrings[bitstrings==0]) - len(bitstrings[bitstrings==1]))/len(bitstrings)

In [10]:
def eval_qnn(x, weights, qubit_list, qvm):
    circuit = create_qnn_circuit(qubit_list, x, weights)
    return run_circuit(circuit, qubit_list, qvm = True)

### Run Predictions

In [11]:
def prediction(X, weights, qubit_list, qvm=True):
    start = time.time()
    results = np.array([])
    i=0
    for x in X:
        i += 1
        if i % 10 == 0:
            print(str(i) + " elements evaluated")
        result = eval_qnn(x, weights, qubit_list, qvm=True)
        result = (result + 1)/2
        results = np.append(results, result)
    print("Time of execution :", time.time()-start)
    return np.where(results > 0.5, 1, 0)

### Predictions Metrics

In [12]:
def metrics_results(predictions, Y_true):
    print(confusion_matrix(Y_true, predictions))
    print(classification_report(Y_true, predictions))

## Predictions

### Quantum Setting

In [13]:
weights = np.load(weights_path, allow_pickle=True)

In [14]:
qubit_list = [35,21,36,22] #Aspen-M-1

### Set to Evaluate

In [15]:
X_sample,Y_sample = create_sample(X_norm,Y,sample_size = 1)

### Evaluation and Metrics on Simulator

In [16]:
results = prediction(X_sample, weights, qubit_list, qvm=True)

10 elements evaluated
20 elements evaluated
30 elements evaluated
40 elements evaluated
50 elements evaluated
60 elements evaluated
70 elements evaluated
80 elements evaluated
90 elements evaluated
100 elements evaluated
Time of execution : 392.6113028526306


In [17]:
metrics_results(results, Y_sample)

[[50  0]
 [ 0 50]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        50
         1.0       1.00      1.00      1.00        50

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



### Evaluation and Metrics on Real QC

In [18]:
results = prediction(X_sample, weights, qubit_list, qvm=False)

10 elements evaluated
20 elements evaluated
30 elements evaluated
40 elements evaluated
50 elements evaluated
60 elements evaluated
70 elements evaluated
80 elements evaluated
90 elements evaluated
100 elements evaluated
Time of execution : 390.843966960907


In [19]:
metrics_results(results, Y_sample)

[[50  0]
 [ 0 50]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        50
         1.0       1.00      1.00      1.00        50

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



## Print Circuit

In [20]:
print_circuit()

H 0
H 1
H 2
H 3
RZ(0) 0
RZ(1) 1
RZ(2) 2
RZ(3) 3
RY(4.846101252746413) 0
RY(4.709903075877427) 1
RY(4.846101252746413) 2
RY(4.709903075877427) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RY(4.846101252746413) 0
RY(4.709903075877427) 1
RY(4.846101252746413) 2
RY(4.709903075877427) 3
H 0
H 1
H 2
H 3
RZ(0) 0
RZ(1) 1
RZ(2) 2
RZ(3) 3
RY(4.846101252746413) 0
RY(4.709903075877427) 1
RY(4.846101252746413) 2
RY(4.709903075877427) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RY(4.846101252746413) 0
RY(4.709903075877427) 1
RY(4.846101252746413) 2
RY(4.709903075877427) 3
DECLARE ro BIT[1]
MEASURE 0 ro[0]

